# Preparing data

In [1]:
# from debias.datasets.mnli import load_hans, load_mnli, tokenize_examples
import py_utils
from load_word_vectors import load_word_vectors
from tokenizer import NltkAndPunctTokenizer
# import tensorflow as tf
tok = NltkAndPunctTokenizer()
STOP_WORDS = frozenset([
  'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your',
  'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her',
  'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
  'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was',
  'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing',
  'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by',
  'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above',
  'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',
  'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few',
  'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than',
  'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', 'd', 'll', 'm', 'o', 're',
  've', 'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma',
  'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn', 'weren', 'won', 'wouldn',
   "many", "how", "de"
])

In [4]:
import pandas as pd
from datasets import load_dataset, load_metric
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
dataset = load_dataset("glue", "qqp")


/home/can/anaconda3/envs/disentangle_project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset glue (/home/can/.cache/huggingface/datasets/glue/qqp/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|██████████| 3/3 [00:00<00:00, 62.17it/s]


In [5]:
dataset['train'].to_pandas()

,question1,question2,label,idx
0,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0,0
1,How do I control my horny emotions?,How do you control your horniness?,1,1
2,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0,2
3,What can one do after MBBS?,What do i do after my MBBS ?,1,3
4,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0,4
...,...,...,...,...
363841,How do I make money flying my drone?,How can I use a dji phantom to make money,1,363841
363842,What can you do with an economics degree?,What jobs can you get with an economics degree?,1,363842
363843,What type of current does a battery produce?,How does a generator work and produce current?,0,363843
363844,Grammar: What is difference between schedule a...,How do I understand the difference between the...,0,363844


In [6]:
dataset['train']=dataset['train'].to_pandas()
dataset['validation']=dataset['validation'].to_pandas()
dataset['test']=dataset['test'].to_pandas()

In [7]:
dataset['train']['question1']=dataset['train']['question1'].apply(tok.tokenize)
dataset['train']['question2']=dataset['train']['question2'].apply(tok.tokenize)
dataset['validation']['question1']=dataset['validation']['question1'].apply(tok.tokenize)
dataset['validation']['question2']=dataset['validation']['question2'].apply(tok.tokenize)
dataset['test']['question1']=dataset['test']['question1'].apply(tok.tokenize)
dataset['test']['question2']=dataset['test']['question2'].apply(tok.tokenize)

In [8]:
dataset['validation']

,question1,question2,label,idx
0,"[Why, are, African, -, Americans, so, beautifu...","[Why, are, hispanics, so, beautiful, ?]",0,0
1,"[I, want, to, pursue, PhD, in, Computer, Scien...","[I, handle, social, media, for, a, non, -, pro...",0,1
2,"[Is, there, a, reason, why, we, should, travel...","[What, are, some, reasons, to, travel, alone, ?]",1,2
3,"[Why, are, people, so, obsessed, with, having,...","[How, can, a, single, male, have, a, child, ?]",0,3
4,"[What, are, some, good, baby, girl, names, sta...","[What, are, some, good, baby, girl, names, sta...",0,4
...,...,...,...,...
40425,"[Why, is, darkness, and, the, colour, black, g...","[Why, does, darkness, represent, evil, ?]",1,40425
40426,"[Why, do, n't, I, feel, uncomfortable, making,...","[Why, am, I, feeling, uncomfortable, to, make,...",1,40426
40427,"[Is, watching, porn, bad, during, your, colleg...","[Is, watching, porn, sin, ?]",1,40427
40428,"[How, do, I, calculate, tension, in, physics, ?]","[How, to, calculate, tension, in, physics, ?]",1,40428


In [9]:
# feature preprocessing
voc = set()
for index, row in dataset['train'].iterrows():
    voc.update(row['question1'])
    voc.update(row['question2'])
words, vecs = load_word_vectors("crawl-300d-2M", voc)

word-vec: 1999996it [00:25, 77464.26it/s] 


In [10]:
import numpy as np
import pickle
w2v = {w: v/np.linalg.norm(v) for w, v in zip(words, vecs)}
with open("w2v_cache", "wb") as f:
    pickle.dump(w2v, f)


   

In [11]:
def is_subseq(needle, haystack):
  l = len(needle)
  if l > len(haystack):
    return False
  else:
    return any(haystack[i:i+l] == needle for i in range(len(haystack)-l + 1))

dataset_to_features = {}
for name in dataset.keys():
    features = []
    for index, row in dataset[name].iterrows():
        h = [x.lower() for x in row.question1]
        p = [x.lower() for x in row.question2]
        p_words = set(p)
        n_words_in_p = sum(x in p_words for x in h)
        fe = {
        "h-is-subseq": is_subseq(h, p),
        "all-in-p": n_words_in_p == len(h),
        "percent-in-p": n_words_in_p / len(h),
        "log-len-diff": np.log(max(len(p) - len(h), 1)),
        "label": row.label
        }    
        h_vecs = [w2v[w] for w in row.question1 if w in w2v]
        p_vecs = [w2v[w] for w in row.question2 if w in w2v]
        if len(h_vecs) > 0 and len(p_vecs) > 0:
          h_vecs = np.stack(h_vecs, 0)
          p_vecs = np.stack(p_vecs, 0)
          # [h_size, p_size]
          similarities = np.matmul(h_vecs, p_vecs.T)
          # [h_size]
          similarities = np.max(similarities, 1)
          similarities.sort()
          fe["average-sim"] = similarities.sum() / len(h)
          fe["min-similarity"] = similarities[0]
          if len(similarities) > 1:
            fe["min2-similarity"] = similarities[1]

        features.append(fe)
    dataset_to_features[name] = pd.DataFrame(features)
    dataset_to_features[name].fillna(0.0, inplace=True)


In [13]:
from sklearn.linear_model import LogisticRegression
train_df = dataset_to_features["train"]
feature_cols = [x for x in train_df.columns if x != "label"]

# class_weight='balanced' will weight the entailemnt/non-entailment examples equally
# C=100 means no regularization
lr = LogisticRegression(multi_class="auto", solver="liblinear",
                    class_weight='balanced', C=100)
lr.fit(train_df[feature_cols].values, train_df.label.values)

LogisticRegression(C=100, class_weight='balanced', solver='liblinear')

In [24]:
for name in dataset_to_features.keys():
  examples = dataset_to_features[name]
  pred = lr.predict_proba(dataset_to_features[name][feature_cols].values).astype(np.float32)
  y = dataset_to_features[name].label.values

  acc = np.mean(y == np.argmax(pred, 1))
  print("%s two-class accuracy: %.4f (size=%d)" % (name, acc, len(examples)))

  dataset[name]["bias_probs"]= pred.tolist()
  temp_json = dataset[name].to_json(orient='records', lines=True)
  with open('qqp.'+name+'.jsonl', 'w') as json_file:
    json_file.write(temp_json)

train two-class accuracy: 0.6696 (size=363846)
validation two-class accuracy: 0.6695 (size=40430)
test two-class accuracy: 0.0000 (size=390965)
